# Comparison of Adjacent TMA sections

**Question:** How does the signal-to-background ratio and staining specificity compare across experimental replicates?

**Samples:** 
- TMA: Purchased from biomax: https://www.biomax.us/tissue-arrays/Breast/BR1506
- Adjacent Sections: BM-Her2N75-15, BM-Her2N75-17, BM-Her2N75-18 (section 16 skipped.)
- Scenes: (i.e. TMA cores) 
  - 17: ER+/HER2+, immune rich. 
  - 49: ER+/HER2+
  - 59: HER2+ immune rich

**Method**: For each stain, pixel intensity was manually thresholded to separate the positive pixels from the negative. Visualizations produced include:
1. Autoscaled overview of stain, with dynamic range of 0 - 99th percentile displayed in legend.
2. Mask of positive pixels, so that masks from adjacent section stains can be compared for specificity.
3. Measure mean intensity in mask area and background, and output values to .csv for downstream analysis.

## Accessing Images

To run this code, first download necessary images from synapse.org

https://www.synapse.org/#!Synapse:syn23644810

https://www.synapse.org/#!Synapse:syn23644811


https://www.synapse.org/#!Synapse:syn23644812

In [1]:
#libraries 
import os
import pandas as pd
import numpy as np
import shutil
import skimage
import matplotlib as mpl
import matplotlib.pyplot as plt
import util

In [2]:
#paths
#os.chdir('/home/groups/graylab_share/OMERO.rdsStore/engje/Data/cycIF_ValidationStudies/cycIF_Validation')
codedir = os.getcwd()

In [4]:
%matplotlib inline
import warnings;
warnings.filterwarnings('ignore')

In [ ]:
#Reproducibility 
#analyze full TMA core

d_crop ={
    'BM-Her2N75-15-Scene-017':(1160,500,4000,4000),
    'BM-Her2N75-17-Scene-017':(1107,1095,4000,4000),
    'BM-Her2N75-18-Scene-017':(2189,2082,4000,4000),

    'BM-Her2N75-15-Scene-049':(2066,879,4000,4000),
    'BM-Her2N75-17-Scene-049':(2009,1397,4000,4000),
    'BM-Her2N75-18-Scene-049':(1148,699,4000,4000),

    'BM-Her2N75-15-Scene-059':(460,950,4000,4000),
    'BM-Her2N75-17-Scene-059':(449,1353,4000,4000),
    'BM-Her2N75-18-Scene-059':(1400,700,4000,4000),

  }

In [ ]:
#load thresholds, rename some columns


df_thresh = pd.read_csv(f'{codedir}/Metadata/BM-Her2N75/SNR_jenny_replicates.csv',index_col=0)
df_thresh['scene_num'] = 'Scene-' + df_thresh.scene.astype('str')
df_thresh['scene'] = df_thresh.tissue
df_thresh['minimum'] = df_thresh.threshold
d_process = {
    'BM-Her2N75':f'{codedir}/Images/tiff',
 }
#results
df_result = pd.DataFrame()

for idx,(s_sample, s_path) in enumerate(d_process.items()):
    df_thresh.index = [item.split('/')[-1] for item in df_thresh.index]
    df_thresh['path'] = f'{s_path}/' + df_thresh.slide + '/' + df_thresh.index
    df_thresh.index = df_thresh.path
    
    for s_marker in sorted(set(df_thresh.marker)):
        
        df_marker = df_thresh[(df_thresh.marker==s_marker)]
        print(f'{s_marker}  {len(df_marker)}')
        #plot the images
        fig = util.array_img(df_marker.sort_values(['scene_num','slide']),s_xlabel='marker',ls_ylabel=['slide','scene_num'],
            s_title='scene',tu_array=(3,len(df_marker)//3),tu_fig=(9,8),cmap='inferno',d_crop=d_crop)
        #fig.savefig(f'{codedir}/Figures/{s_sample}/Replicate_TissueLoss_Background_{s_marker}.png')
        plt.close(fig)
        df_marker_thresh,d_mask = util.thresh_meanint(df_marker,d_crop)
        df_result =df_result.append(df_marker_thresh)
        #plot the mask
        fig, ax = plt.subplots(3,len(df_marker)//3,figsize=(10,10))
        ax=ax.ravel()
        for idx,(s_index, a_mask) in enumerate(d_mask.items()):
            ax[idx].imshow(a_mask)
            ax[idx].set_title(df_thresh.loc[s_index,'scene'])
        #fig.savefig(f'{codedir}/Figures/{s_sample}/Replicate_Thresholding_{s_marker}.png')
